In [ ]:
import pandas as pd

data = pd.read_csv('./imagenette2/noisy_imagenette.csv')
data.head(10)

,path,noisy_labels_0,noisy_labels_1,noisy_labels_5,noisy_labels_25,noisy_labels_50,is_valid
0,train/n02979186/n02979186_9036.JPEG,n02979186,n02979186,n02979186,n02979186,n02979186,False
1,train/n02979186/n02979186_11957.JPEG,n02979186,n02979186,n02979186,n02979186,n03000684,False
2,train/n02979186/n02979186_9715.JPEG,n02979186,n02979186,n02979186,n03417042,n03000684,False
3,train/n02979186/n02979186_21736.JPEG,n02979186,n02979186,n02979186,n02979186,n03417042,False
4,train/n02979186/ILSVRC2012_val_00046953.JPEG,n02979186,n02979186,n02979186,n02979186,n03394916,False
5,train/n02979186/n02979186_10568.JPEG,n02979186,n02979186,n02979186,n02979186,n03000684,False
6,train/n02979186/n02979186_2745.JPEG,n02979186,n02979186,n02979186,n03394916,n02979186,False
7,train/n02979186/n02979186_3529.JPEG,n02979186,n02979186,n02979186,n02979186,n03417042,False
8,train/n02979186/n02979186_10756.JPEG,n02979186,n02979186,n02979186,n02979186,n03445777,False
9,train/n02979186/n02979186_7058.JPEG,n02979186,n02979186,n02979186,n02979186,n02979186,False
